# Описание проекта

Вы работаете в интернет-магазине «Стримчик», который продаёт по всему миру компьютерные игры. Из открытых источников доступны исторические данные о продажах игр, оценки пользователей и экспертов, жанры и платформы (например, Xbox или PlayStation). Вам нужно выявить определяющие успешность игры закономерности. Это позволит сделать ставку на потенциально популярный продукт и спланировать рекламные кампании.


Перед вами данные до 2016 года. Представим, что сейчас декабрь 2016 г., и вы планируете кампанию на 2017-й. Нужно отработать принцип работы с данными. Неважно, прогнозируете ли вы продажи на 2017 год по данным 2016-го или же 2027-й — по данным 2026 года.


В наборе данных попадается аббревиатура ESRB (Entertainment Software Rating Board) — это ассоциация, определяющая возрастной рейтинг компьютерных игр. ESRB оценивает игровой контент и присваивает ему подходящую возрастную категорию, например, «Для взрослых», «Для детей младшего возраста» или «Для подростков».

## Описание данных

- Name — название игры
- Platform — платформа
- Year_of_Release — год выпуска
- Genre — жанр игры
- NA_sales — продажи в Северной Америке (миллионы проданных копий)
- EU_sales — продажи в Европе (миллионы проданных копий)
- JP_sales — продажи в Японии (миллионы проданных копий)
- Other_sales — продажи в других странах (миллионы проданных копий)
- Critic_Score — оценка критиков (максимум 100)
- User_Score — оценка пользователей (максимум 10)
- Rating — рейтинг от организации ESRB (англ. Entertainment Software Rating Board). Эта ассоциация определяет рейтинг компьютерных игр и присваивает им подходящую возрастную категорию.

## Задачи

1. Просмотреть данные, изучить информацию, обработать пропуска, привести к нужному типу данных
2. Посмотреть сколько игр выпускалось в разные годы. Посмотреть, как менялись продажи по платформам. 
3. Посмотреть, как влияют на продажи внутри одной популярной платформы отзывы пользователей и критиков. Посмотреть на общее распределение игр по жанрам
4. Определить для пользователя каждого региона самые популярные платформы и жанры. Посмотреть влияет ли поставленный рейтинг на продажи
5. Проверить гипотезы

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from scipy import stats as st
import seaborn as sns
from IPython.display import display
import numpy as np

In [2]:
data = pd.read_csv('/datasets/games.csv')
display(data)

FileNotFoundError: [Errno 2] No such file or directory: '/datasets/games.csv'

In [ ]:
data.info()

Много пропущенных значений в столбцах оценок критиков и пользователей, а также есть пропущенные значения с рейтингом игры. 

## Шаг 2. Подготовьте данные

- Замените названия столбцов (приведите к нижнему регистру);
- Преобразуйте данные в нужные типы. Опишите, в каких столбцах заменили тип данных и почему;
- Обработайте пропуски при необходимости:
   - Объясните, почему заполнили пропуски определённым образом или почему не стали это делать;
   - Опишите причины, которые могли привести к пропускам;
   - Обратите внимание на аббревиатуру 'tbd' в столбцах с рейтингом. Отдельно разберите это значение и опишите, как его обработать;
- Посчитайте суммарные продажи во всех регионах и запишите их в отдельный столбец.

In [ ]:
#заменим названия столбцов, приведя к нижнему регистру
data.columns = data.columns.str.lower()

In [ ]:
data

In [ ]:
data.isna().sum()

Заполним оценки нулями, так как не известны причины отсутствия рейтинга. Даже если предположить, что на одной платформе игра вышла крутой, то совершенно необязательно она выйдет крутой на другой платформе. Также заменим год выпуска игры на 0, так как доля этих значений мала и данные не исказятся плюс заменять на рандомные года не совсем точно

In [ ]:
#заменим пропущенные значения 
data['name'] = data['name'].fillna('no game')
data['year_of_release'] = data['year_of_release'].fillna(0)
data['genre'] = data['genre'].fillna('no genre game')
data['critic_score'] = data['critic_score'].fillna(0)
data['user_score'] = data['user_score'].fillna(0)
data['rating'] = data['rating'].fillna(0)

In [ ]:
data.isna().sum()

In [ ]:
data['user_score'].unique()

Аналогично заменим значение 'tbd' на 0, чтобы в дальнейшем изменить на тип данных с плавающей запятой

In [ ]:
data.loc[(data['user_score'] == 'tbd') | (data['user_score'].isna()), 'user_score'] = 0

In [ ]:
#заменим столбцы на нужный тип данных
data['year_of_release'] = data['year_of_release'].astype('int')
data['user_score'] = data['user_score'].astype('float')
data['critic_score'] = data['critic_score'].astype('int')

In [ ]:
#найдем суммарное значение по продажам
data['total_sales']= data['na_sales']+data['eu_sales']+data['jp_sales']+data['other_sales']

In [ ]:
data.head()

In [ ]:
data.info()
data.describe()

## Шаг 3. Проведите исследовательский анализ данных

- Посмотрите, сколько игр выпускалось в разные годы. Важны ли данные за все периоды?
- Посмотрите, как менялись продажи по платформам. Выберите платформы с наибольшими суммарными продажами и постройте распределение по годам. За какой характерный срок появляются новые и исчезают старые платформы?
- Возьмите данные за соответствующий актуальный период. Актуальный период определите самостоятельно в результате исследования предыдущих вопросов. Основной фактор — эти данные помогут построить прогноз на 2017 год.
- Не учитывайте в работе данные за предыдущие годы.
- Какие платформы лидируют по продажам, растут или падают? Выберите несколько потенциально прибыльных платформ.
- Постройте график «ящик с усами» по глобальным продажам игр в разбивке по платформам. Опишите результат.
- Посмотрите, как влияют на продажи внутри одной популярной платформы отзывы пользователей и критиков. Постройте диаграмму рассеяния и посчитайте корреляцию между отзывами и продажами. Сформулируйте выводы.
- Соотнесите выводы с продажами игр на других платформах.
- Посмотрите на общее распределение игр по жанрам. Что можно сказать о самых прибыльных жанрах? Выделяются ли жанры с высокими и низкими продажами?

In [ ]:
# составим сводную таблицу, чтобы посмотреть сколько игр выпускалось в разные годы
data_year = data.pivot_table(index='year_of_release', values='total_sales', aggfunc='count')
data_year.columns = ['count']

# строим график
ax = plt.subplots(figsize=(22, 8))
ax = sns.barplot(x=data_year.index, y="count", data=data_year)
plt.title('Количество выпускаемых игр по годам')

Начиная с 1994 года количество игр начало значительно увеличиваться. Пик пришел на 2008-2009 года, далее пошел спад по выпускаемым играм

In [ ]:
#Посмотрите, как менялись продажи по платформам. 
#Выберите платформы с наибольшими суммарными продажами и постройте распределение по годам. За какой характерный срок появляются новые и исчезают старые платформы?
data_sales = data.pivot_table(index='year_of_release', columns='platform', values='total_sales', aggfunc='sum')
data_sales.fillna(0, inplace=True)

In [ ]:
# соберем список из количества лет жизни каждой платформы
platform_year=[]
for platform in data_sales.columns:
    platform_year.append(data_sales[platform].ne(0).value_counts().loc[data_sales[platform].ne(0).value_counts().index == True].values[0])

In [ ]:
platform_year = pd.DataFrame(platform_year, columns=['active_years'])
platform_year.describe()

Как мы видим, средняя продолжительность игры 8 лет

In [ ]:
#составим сводную таблицу с топ-10 платформами
top10 = data.pivot_table(index='platform', values='total_sales', aggfunc='sum').nlargest(10, 'total_sales')
top10

Далее, рассмотрим разбиение продаж по каждой платформе из топ-10 по годам, предварительно отбросим неинтересующий нас период до 1994 года.

In [ ]:
for platform in data_sales.columns:
    if platform not in top10.index:
        data_sales.drop(platform, axis=1, inplace=True)
timeseries = data_sales[data_sales.index >= 1994]
timeseries

In [ ]:
ax = timeseries.plot(y=['DS','GBA','PC','PS','PS2','PS3','PS4','PSP','Wii','X360'],kind='bar', legend = 10, figsize=(22, 12))
plt.title('Изменение продаж по платформам')

Возьмем за актуальный период можно 2013-2016 гг. Это период активной фазы платформы PS4, существование остальных платформ подходит к концу к этому периоду, а значит на 2017 год их продажи могут быть еще меньше

In [ ]:
timeseries = timeseries[timeseries.index >= 2013]

In [ ]:
ax = timeseries.plot(y=['DS','GBA','PC','PS','PS2','PS3','PS4','PSP','Wii','X360'],kind='bar', legend = 10, figsize=(22, 12))
plt.title('Изменение продаж по платформам с 2013 года')

Как было сказано выше, в своей активной коммерческой фазе пребывает платформа PS4, тогда как лидеры по продажам 2013 года: PS3 и X360, — к 2015-2016 гг. уже исчерпали свой коммерческий ресурс.

In [ ]:
#Постройте график «ящик с усами» по глобальным продажам игр в разбивке по платформам.
actual_data = data.query('year_of_release >= 2013')

# строим график
sns.catplot(x="platform", 
            y='total_sales', 
            kind="box", 
            data=actual_data, 
            orient='v', 
            height=6, 
            aspect=2.7)
plt.ylim(0, 2)

In [ ]:
actual_data.pivot_table(index='platform', values='total_sales', aggfunc='sum').nlargest(10, 'total_sales')

К лидерам по продажам за актуальный период (2013-2016 гг.) относятся PS4, PS3 и XOne. 

In [ ]:
#Посмотрите, как влияют на продажи внутри одной популярной платформы отзывы пользователей и критиков. 
#Постройте диаграмму рассеяния и посчитайте корреляцию между отзывами и продажами
#Посмотрим на примере PS4
ps4 = actual_data.query('platform == "PS4" and (user_score != 0) and (critic_score != 0)')
for column in ['user_score', 'critic_score']:
    print(round(ps4[column].corr(ps4['total_sales']), 2))
    sns.relplot(x=column, y="total_sales", data=ps4)

Отзывы пользователей никак не влияют на продажи игр, в то время как отзывы критиков имеют некоторый вес в продажах: коэффициенты корреляции -0,03 и 0,41 соответственно.

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#Соотнесите выводы с продажами игр на других платформах.
#Посмотрим на примере первых 5 из рейтинга по продажам, не считая PS4
for platform in ['PS3','XOne' ,'3DS', 'X360']:
    print('\n{0}:'.format(platform))
    for column in ['user_score', 'critic_score']:
        temp = actual_data.query('platform == @platform and (user_score != 0) and (critic_score != 0)')
        print('Коэффициент корреляции {0} и total_sales: {1}'.\
              format(column, round(temp[column].corr(temp['total_sales']), 2)))
        sns.relplot(x=column, y="total_sales", data=temp)
        _ = plt.title(platform)

Данные показали, что на других платформах отзывы пользователей имеют не сильное влияние на продажи. А вот оценки критиков оказывают среднее влияние (коэффициент корреляции ~0.3 - 0.4). Оценки пользователей могут не оказывать влияние, т.к. пользователи в основном ориентируются на мнение критиков

In [ ]:
#Посмотрите на общее распределение игр по жанрам. 
sns.catplot(x="genre", 
            y='total_sales', 
            kind="box", 
            data=actual_data, 
            orient='v', 
            height=6, 
            aspect=2.7)
plt.ylim(0, 2)

Как мы видим, shooter по средним продажам лидирует по сравнению с остальными жанрами игр, а sports, напротив, по этому критерию отстает, однако, является в целом популярным жанром из-за наибольшего объема продаж. Advanture и puzzle, как правило, не пользуются большой популярностью: их абсолютные и средние продажи одни из самых низких среди всех жанров.

## Шаг 4. Составьте портрет пользователя каждого региона

Определите для пользователя каждого региона (NA, EU, JP):
 - Самые популярные платформы (топ-5). Опишите различия в долях продаж.
 - Самые популярные жанры (топ-5). Поясните разницу.
 - Влияет ли рейтинг ESRB на продажи в отдельном регионе?

In [ ]:
#самые популярные платформы
na_sales = actual_data.pivot_table(index='platform', values='na_sales', aggfunc='sum').nlargest(5, 'na_sales')
na_sales.columns = ['sales']

eu_sales = actual_data.pivot_table(index='platform', values='eu_sales', aggfunc='sum').nlargest(5, 'eu_sales')
eu_sales.columns = ['sales']

jp_sales = actual_data.pivot_table(index='platform', values='jp_sales', aggfunc='sum').nlargest(5, 'jp_sales')
jp_sales.columns = ['sales']

In [ ]:
for column in ['na_sales', 'eu_sales', 'jp_sales']:
    region = actual_data.pivot_table(index='platform', values=column, aggfunc='sum').\
    sort_values(column, ascending=False)
    region['ratio'] = round(region[column] / region[column].sum(), 2)
    print(region.nlargest(5, column))

Интересно, что если в Америке и Европе набор топ-5 платформ одинаковый и различается лишь долями, то в Японии топ-5 платформ кардинально другой.

Если в Америке и Европе самой популярной платформой является PS4, то в Японии эта платформа лишь четвертая, а на первом месте в стране восходящего солнца платформа 3DS, которая на Западе, напротив, занимает 5 место по популярности. Кроме того, в Японии в топ-5 вошла PSV

In [ ]:
#самые популярные жанры
for column in ['na_sales', 'eu_sales', 'jp_sales']:
    region = actual_data.pivot_table(index='genre', values=column, aggfunc='sum').\
    sort_values(column, ascending=False)
    region['ratio'] = round(region[column] / region[column].sum(), 2)
    print(region.nlargest(5, column))

В Америке и Европы предпочтения по жанрам опять схожи. Action занимет первые места про продажам (хоть и самым популярным жанром оказался shooter), который в данных странах лишь на 3 месте. 

В Японии опять же первое место знамает жанр ролевых игр, а самый популярный жанр оказался на 5 месте.

Рейтинги ESRB:

- E - от 6 лет
- E10+ - от 10 лет
- T - от 13 лет
- M - от 17 лет

In [ ]:
#рейтинг
for column in ['na_sales', 'eu_sales', 'jp_sales']:
    region = (actual_data.pivot_table(index='rating', values=column, aggfunc='sum').
              sort_values(column, ascending=False)
             )
    region['ratio'] = round(region[column] / region[column].sum(), 2)
    print(region)

Запад и Восток вновь отличаются по популярности рейтингов: на Западе популярностью пользуются игры для лиц старше 17 лет, тогда как на Востоке первое место по продажам принадлежит играм с неуказанным рейтингом (Из-за того, что в Японии своя система рейтингов, а данные европейские). Похожая ситуация и в Северной Америке, игры с отличным от европейской классификации рейтингом занимают 2 по популярности место. По общей классификации в Японии лидируют игры с рейтингом для подростков, которые наоброт на Западе занимают последнее место

## Шаг 5. Проверьте гипотезы

- Средние пользовательские рейтинги платформ Xbox One и PC одинаковые;
- Средние пользовательские рейтинги жанров Action (англ. «действие», экшен-игры) и Sports (англ. «спортивные соревнования») разные.

Будем использовать t-критерий Стьюдента, так как сравниваемые выборки независимы между собой.

Сформулируем гипотезы:
- Н0: Средние пользовательские рейтинги платформ Xbox One и PC одинаковые
- Н1: Средние пользовательские рейтинги платформ Xbox One и PC различаются


In [ ]:
xbox_one = actual_data.query('platform == "XOne"')['user_score']
pc = actual_data.query('platform == "PC"')['user_score']

In [ ]:
alpha = .05

In [ ]:
print('Дисперсия Xbox One', np.var(xbox_one))
print('Дисперсия PC', np.var(pc))

In [ ]:
results = st.ttest_ind(xbox_one, pc, equal_var = False)
    
print('p-значение: ', results.pvalue)
    
if (results.pvalue < alpha):
    print("Отвергаем нулевую гипотезу")
else:
    print("Не получилось отвергнуть нулевую гипотезу") 

Статистический тест показал, что гипотезу о равенстве среднего пользовательского рейтинга для XOne и PC не получилось отвергнуть (p-значение 0.24400). 

Сформулируем гипотезы:
- Н0: Средние пользовательские рейтинги жанров Action и Sports разные
- Н1: Средние пользовательские рейтинги жанров Action и Sports одинаковые

In [ ]:
action = actual_data.query('genre == "Action"')['user_score']
sports = actual_data.query('genre == "Sports"')['user_score']

In [ ]:
alpha = .05

In [ ]:
print('Дисперсия Action', np.var(action))
print('Дисперсия Sports', np.var(sports))

In [ ]:
results = st.ttest_ind(action, sports, equal_var = False)
    
print('p-значение: ', results.pvalue)
    
if (results.pvalue < alpha):
    print("Отвергаем нулевую гипотезу")
else:
    print("Не получилось отвергнуть нулевую гипотезу") 

Статистический тест показал, что гипотезу о равенстве среднего пользовательского рейтинга жанров Action и Sports не получилось отвергнуть (p-значение 0.05180).

## Шаг 6. Напишите общий вывод

Анализ показал, что потенциально успешный продукт должен обладать рядом характеристик:

1. Игра должна быть разработана для платформ PS4 или XOne, если речь идет о западном рынке, и — для платформы 3DS, если игру планируется продавать в Японии.
2. Жанр: Action или Shooter для Запада, Role-Playing — для Японии.
3. Рейтинг ESRB: либо от 17 лет (для Америки и Европы) или от 13 лет (для Японии). Рейтинг от 6 лет и старше также будет хорошим выбором, но тогда набор жанров сужается. Также стоит изучить иную классификацию рейтингов, отличной от Западной, т.к. не все игры имеют рейтинг ESRB
4. Ориентироваться на оценки пользователей нет смысла, на продажи это не оказывает существенного влияния. Однако отзывы критиков не стоит упускать из внимания, их отзывы имеют значение. 